***Challenge 1***

Here the goal is to train on 25 samples. In this preliminary testbed the evaluation will be done on a 2000 sample validation set. Note in the end the final evaluation will be done on the full CIFAR-10 test set as well as potentially a separate dataset. The validation samples here should not be used for training in any way, the final evaluation will provide only random samples of 25 from a datasource that is not the CIFAR-10 training data.

Feel free to modify this testbed to your liking, including the normalization transformations etc. Note however the final evaluation testbed will have a rigid set of components where you will need to place your answer. The only constraint is the data. Refer to the full project instructions for more information.


Setup training functions. Again you are free to fully modify this testbed in your prototyping within the constraints of the data used. You can use tools outside of pytorch for training models if desired as well although the torchvision dataloaders will still be useful for interacting with the cifar-10 dataset.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import CIFAR10
from torchvision.transforms import transforms
from torch.utils.data import DataLoader, Dataset
import random
import numpy as np
from torchvision.models import resnet18

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Load CIFAR-10 data
def load_cifar10_data():
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
    return train_dataset

class CustomCIFAR10(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

def sample_task(dataset, num_samples_per_class=5, num_query_per_class=15, used_indices=None):
    if used_indices is None:
        used_indices = set()

    chosen_classes = random.sample(range(10), 2)  # Select two random classes
    class_map = {chosen_classes[i]: i for i in range(2)}
    support_data = []
    query_data = []
    support_labels = []
    query_labels = []
    indices_per_class = {class_map[chosen_classes[0]]: [], class_map[chosen_classes[1]]: []}

    for idx, (image, label) in enumerate(dataset):
        if label in chosen_classes and idx not in used_indices:
            class_label = class_map[label]
            if len(indices_per_class[class_label]) < num_samples_per_class + num_query_per_class:
                if len(indices_per_class[class_label]) < num_samples_per_class:
                    support_data.append(image)
                    support_labels.append(class_label)
                else:
                    query_data.append(image)
                    query_labels.append(class_label)
                indices_per_class[class_label].append(idx)
                used_indices.add(idx)
            if all(len(indices_per_class[c]) == num_samples_per_class + num_query_per_class for c in indices_per_class):
                break

    return CustomCIFAR10(support_data, support_labels), CustomCIFAR10(query_data, query_labels)


In [ ]:
def euclidean_dist(x, y):
    return torch.cdist(x, y)

def prototypical_loss(support_features, query_features, support_labels, query_labels):
    # Calculate prototypes as the mean of support features by class
    unique_labels = torch.unique(support_labels)
    prototypes = torch.stack([support_features[support_labels == label].mean(0) for label in unique_labels])

    # Calculate distances from query features to prototypes
    dists = euclidean_dist(query_features, prototypes)

    # Use log_softmax and negative log likelihood loss
    log_p_y = torch.nn.functional.log_softmax(-dists, dim=1)
    loss_val = torch.nn.functional.nll_loss(log_p_y, query_labels)
    return loss_val



def meta_train(model, dataset, device, epochs=50, tasks_per_epoch=5, num_support=5, num_query=15):
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(epochs):
        model.train()
        used_indices = set()
        total_loss = 0
        for _ in range(tasks_per_epoch):
            support_set, query_set = sample_task(dataset, num_samples_per_class=num_support, num_query_per_class=num_query, used_indices=used_indices)
            support_loader = DataLoader(support_set, batch_size=len(support_set), shuffle=True)
            query_loader = DataLoader(query_set, batch_size=len(query_set), shuffle=True)

            support_data, support_labels = next(iter(support_loader))
            query_data, query_labels = next(iter(query_loader))

            support_data, support_labels = support_data.to(device), support_labels.to(device)
            query_data, query_labels = query_data.to(device), query_labels.to(device)

            optimizer.zero_grad()
            support_features = model(support_data)
            query_features = model(query_data)
            loss = prototypical_loss(support_features, query_features, support_labels, query_labels)  # Updated to remove the extra argument
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f'Epoch {epoch + 1}, Average Loss: {total_loss / tasks_per_epoch}')


In [ ]:
def meta_test(model, dataset, device, num_samples_per_class=5, num_support=3, num_query=2, num_tasks=10):
    accuracies = []
    model.eval()  # Set the model to evaluation mode
    used_indices = set()  # Initialize here to track indices across tasks

    with torch.no_grad():
        for _ in range(num_tasks):
            # Sample a new task
            support_set, query_set = sample_task(dataset, num_samples_per_class=num_samples_per_class, num_query_per_class=num_query, used_indices=used_indices)
            support_loader = DataLoader(support_set, batch_size=len(support_set), shuffle=True)
            query_loader = DataLoader(query_set, batch_size=len(query_set), shuffle=False)

            # Get data for support and query sets
            support_data, support_labels = next(iter(support_loader))
            query_data, query_labels = next(iter(query_loader))

            support_data, support_labels = support_data.to(device), support_labels.to(device)
            query_data, query_labels = query_data.to(device), query_labels.to(device)

            # Calculate prototypes
            support_features = model(support_data)
            unique_labels = torch.unique(support_labels)
            prototypes = torch.stack([support_features[support_labels == label].mean(0) for label in unique_labels])

            # Evaluate on query data
            query_features = model(query_data)
            dists = euclidean_dist(query_features, prototypes)
            _, predicted = torch.min(dists, 1)

            # Calculate accuracy
            correct = (predicted == query_labels).sum().item()
            total = query_labels.size(0)
            accuracy = correct / total
            accuracies.append(accuracy)

    average_accuracy = np.mean(accuracies)
    print(f'Average Test Accuracy on new tasks: {average_accuracy * 100:.2f}%')
    return average_accuracy*100


# Prepare a test dataset (assuming classes not seen during training)
def load_cifar10_test_data():
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)
    return test_dataset


In [ ]:
# Modify ResNet to fit the small data scenario
# def create_small_resnet(num_classes=2, dropout_rate=0.5):
#     model = resnet18(pretrained=False)
#     num_features = model.fc.in_features
#     model.fc = nn.Sequential(
#         nn.Dropout(dropout_rate),
#         nn.Linear(num_features, num_classes)
#     )
#     return model

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layers = nn.ModuleList()

        self.layers+=[nn.Conv2d(3, 16,  kernel_size=3) ,
                      nn.ReLU(inplace=True)]
        self.layers+=[nn.Conv2d(16, 16,  kernel_size=3, stride=2),
                      nn.ReLU(inplace=True)]
        self.layers+=[nn.Conv2d(16, 32,  kernel_size=3),
                      nn.ReLU(inplace=True)]
        self.layers+=[nn.Conv2d(32, 32,  kernel_size=3, stride=2),
                      nn.ReLU(inplace=True)]
        self.fc = nn.Linear(32*5*5, 2)
    def forward(self, x):
        for i in range(len(self.layers)):
          x = self.layers[i](x)
        x = x.view(-1, 32*5*5)
        x = self.fc(x)
        return x

The below tries  2 random problem instances. In your development you may choose to prototype with 1 problem instances but keep in mind for small sample problems the variance is high so continously evaluating on several subsets will be important.

In [ ]:
from numpy.random import RandomState
import numpy as np
import torch.optim as optim
from torch.utils.data import Subset
import time



from torchvision import datasets, transforms
normalize = transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))

transform_val = transforms.Compose([transforms.ToTensor(), normalize]) #careful to keep this one same
transform_train = transforms.Compose([transforms.ToTensor(), normalize])

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

##### Cifar Data
cifar_data = datasets.CIFAR10(root='.',train=True, transform=transform_train, download=True)

#We need two copies of this due to weird dataset api
cifar_data_val = datasets.CIFAR10(root='.',train=True, transform=transform_val, download=True)


accs = []
times = []


for seed in range(1, 5):

  train_dataset = load_cifar10_data()
  test_dataset = load_cifar10_test_data()

  model = Net().to(device)
  # model = create_small_resnet().to(device)
  start_time = time.time()
  meta_train(model, train_dataset, device)
  end_time = time.time()
  times.append(end_time - start_time)

  accuracy = meta_test(model, test_dataset, device, num_samples_per_class=5, num_support=3, num_query=2, num_tasks=5)

  accs.append(accuracy)

times = np.array(times)
accs = np.array(accs)
print('Acc over 5 instances: %.2f +- %.2f'%(accs.mean(),accs.std()))
print(f"Average Time over 5 instances: {times.mean()} +-{times.std()}")



100%|██████████| 170498071/170498071 [00:13<00:00, 12961755.56it/s]


Extracting ./cifar-10-python.tar.gz to .
Files already downloaded and verified


100%|██████████| 170498071/170498071 [00:13<00:00, 12835544.42it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Epoch 1, Average Loss: 0.6927608847618103
Epoch 2, Average Loss: 0.687661325931549
Epoch 3, Average Loss: 0.6747776985168457
Epoch 4, Average Loss: 0.6835201025009155
Epoch 5, Average Loss: 0.6658149838447571
Epoch 6, Average Loss: 0.6117475986480713
Epoch 7, Average Loss: 0.6928929686546326
Epoch 8, Average Loss: 0.6321070253849029
Epoch 9, Average Loss: 0.6773884057998657
Epoch 10, Average Loss: 0.656679904460907
Epoch 11, Average Loss: 0.6173013389110565
Epoch 12, Average Loss: 0.6800037860870362
Epoch 13, Average Loss: 0.653512179851532
Epoch 14, Average Loss: 0.6497628688812256
Epoch 15, Average Loss: 0.6122658610343933
Epoch 16, Average Loss: 0.5557719737291336
Epoch 17, Average Loss: 0.6887284934520721
Epoch 18, Average Loss: 0.5511030942201615
Epoch 19, Average Loss: 0.5957540571689606
Epoch 20, Average Loss: 0.43481029868125914
Epoch 21, Average Loss: 0.6259779453277587
Epoch 22, Average L